In [1]:
import os, glob
import numpy as np
import pandas as pd
from scipy import stats
from datetime import date, time, datetime
from datetime import timedelta
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,  accuracy_score
from keras.models import load_model
from sklearn.metrics import mean_squared_error
w_list= glob.glob("../result/merge_weather/*.csv")
p_list = glob.glob("../result/flower_price_byweek/*")
select_t = [(1,0),(3,2),(2,1),(2,3),(2,4)]
print(w_list)

Using TensorFlow backend.


['../result/merge_weather\\467480_嘉義.csv', '../result/merge_weather\\C0F9L0_后里.csv', '../result/merge_weather\\C0K490_古坑.csv', '../result/merge_weather\\C0X060_下營.csv']


In [106]:
class fp_pre:
    def __init__(self,fn_weather, fn_price):
        self.r_data = pd.read_csv(fn_weather, encoding="utf-8")
        self.lc_d = pd.read_csv("../lunar_done_new.csv", encoding="utf-8")
        self.df_pbw = pd.read_csv(fn_price, encoding="utf-8")
        self.d_data = self.r_data["Date"]
        self.flower_name = fn_price.split("\\")[-1].split("_")[0]
        self.station_name = fn_weather.split("\\")[-1].split("_")[0]
    
    def market_select(self, market):
        self.df_pbw = self.df_pbw.groupby("market").get_group(market)
    
    def prep_rdata(self):
        r_data = self.r_data
        col_fix =  ['T.Max', 'T.Min', 'Precp','Temperature', 'RH', 'StnPres', 'WS', 'WSGust']
        for col in col_fix:
            old = r_data[col]
            new = []
            for i in range(len(old)):
                try:
                    new.append(float(old[i]))
                    tmp = float(old[i])
                except ValueError:
                    new.append(tmp)
            r_data[col] = new
        d_tmp = r_data['T.Max'] - r_data['T.Min']
        r_data["d_tmp"]= d_tmp
        drop_c =["Date",'T.Max', 'T.Min']
        r_data = r_data.drop(columns=drop_c)
        
        def to_zscore(df):
            col_x = df.columns.to_list()
            mean = df.mean(axis=0)
            std = df.std(axis=0)
            for i in range(len(col_x)):
                df[col_x[i]]=(df[col_x[i]]-mean[i])/std[i]
            return  df , [list(mean), list(std)]
        r_data, self.recordz = to_zscore(r_data)
        self.r_data = r_data
        self.r_data_pre_done = r_data
    
    
    def join_lunar(self):
        self.r_data["Date"] = self.d_data
        self.r_data = self.r_data.join(self.lc_d.set_index("date"), on="Date")
        self.r_data = self.r_data.drop(columns="Date")
        
    
    
    def shift_rdata(self, shift1, shift2):
        dfs =[]
        arr_all =[]
        if shift2==0:
            df_s = self.r_data.copy()
        else:
            df_s = self.r_data.copy()
            df_s = df_s.shift(periods=shift2)
        arr_all = np.array(df_s)

        for i in range(1,shift1):
            tp = np.array(df_s.shift(periods=i))
            arr_all = np.concatenate((arr_all, tp), axis=1)
        df_all = pd.DataFrame(arr_all)
        df_all["date"] = self.d_data
        df_all = df_all.dropna()
        df_all = df_all.reset_index()
        df_all = df_all.drop(columns="index")
        self.d_data = df_all["date"]
        self.r_data = df_all
    
    
    def cutyw_rdata(self):
        def trans_to_y_w(d_date):
            year = int(d_date.split("-")[0])
            d_day = date(year,int(d_date.split("-")[1]), int(d_date.split("-")[2]))- date(year, 1, 1)
            d_w = 1+ (d_day.days // 7)
            if d_w ==53:
                d_w = 52
            result =  str(year) + "_" + str(d_w)

            if d_day.days % 7 ==6:
                return result
            else:
                return np.nan
        y_w = []
        for i in range(len(self.d_data)):
            y_w.append(trans_to_y_w(self.d_data[i]))
        self.r_data["y_w"] = y_w
        self.r_data_n_drop = self.r_data.copy()
        self.r_data = self.r_data.dropna()
        self.r_data = self.r_data.reset_index(drop=True)
        self.cy_data = self.d_data

    
    
    def join_pbw_r_data(self):
        df_join = self.df_pbw.join(self.r_data.set_index("y_w"), on="y_w")
        df_join = df_join.dropna()
        df_join = df_join.reset_index(drop=True)
#         df_join = df_join.drop(columns="index")
        self.d_data = df_join["date"]
        self.r_data = df_join
    
    
    def dummy_market(self,d_t_dummy):
        tmp_dum = pd.DataFrame()
        for col in d_t_dummy:
            tmp_dum[col] = self.r_data[col]
            dummy = pd.get_dummies(tmp_dum[col])
            self.r_data = pd.concat([self.r_data, dummy], axis=1)
        self.r_data = self.r_data.drop(columns=d_t_dummy)
        self.x = self.r_data
        self.dummy = dummy
#         [,"year", "week","w_avg", "w_sale", "date", "y_w", "price_diff"]
    
    
    def drop_x_data(self,drop_c):
        self.x_beforedrop = self.x
        self.y_raw_price = self.x["w_avg"]
        self.x = self.x.drop(columns=drop_c)
        self.input_units = self.x.shape[1]
        
    
    
    def all_prep(self, shift1, shift2,d_t_dummy):
        self.prep_rdata()
        self.join_lunar()
        self.shift_rdata(shift1, shift2)
        self.shift1 = shift1
        self.shift2 = shift2
        self.cutyw_rdata()
        self.join_pbw_r_data()
        self.dummy_market(d_t_dummy)
        self.extract_xy()
        
    def all_prep_market_select(self, shift1, shift2,d_t_dummy,market):
        self.market_select(market)
        self.prep_rdata()
        self.join_lunar()
        self.shift_rdata(shift1, shift2)
        self.shift1 = shift1
        self.shift2 = shift2
        self.cutyw_rdata()
        self.join_pbw_r_data()
        self.dummy_market(d_t_dummy)
        self.extract_xy()
    
    def all_prep_nolunar(self, shift1, shift2,d_t_dummy):
        self.prep_rdata()
#         self.join_lunar()
        self.shift_rdata(shift1, shift2)
        self.shift1 = shift1
        self.shift2 = shift2
        self.cutyw_rdata()
        self.join_pbw_r_data()
        self.dummy_market(d_t_dummy)
        self.extract_xy()
    
    def extract_xy(self):
#     y_date = df_join["date"]
        self.y_yw = self.r_data["y_w"]
        self.y_raw = np.array(self.r_data["price_diff"])
        self.y_diff_d = np.array(self.r_data["w_avg"].diff())
        
    def split_y_to_class(self, input_y, list_y_class=[0]):
        y_class = []
        for i in range(len(input_y)):
            c = 0
            y = input_y[i]
            for j in range(len(list_y_class)):
                if y > list_y_class[j]:
                    c = c +1
            y_class.append(c)
#         print((np.bincount(y_class), list_y_class))
        self.output_class = len(np.bincount(y_class))
        return np.array(y_class)
    
    def y_to_class(self, list_y_class=[-0.1,0,0.1]):
        y_raw = self.y_raw
        self.y_list_class = list_y_class
        y_class = []
        for i in range(len(y_raw)):
            c = 0
            y = y_raw[i]
            for j in range(len(list_y_class)):
                if y > list_y_class[j]:
                    c = c +1
            y_class.append(c)
        self.y_class = np.array(y_class)
        self.y_freq = (np.bincount(y_class), list_y_class)
        self.output_class =len(np.bincount(y_class))
    
    
    def split_data(self, year,year_s=0):
        y_w = self.y_yw
        y_raw = self.y_raw
        y_year = []
        for  i in range(len(y_w)):
            y_year.append(int(y_w[i].split("_")[0]))
        y_year = np.array(y_year)
        self.split_info = np.unique(np.array(y_year), return_counts=True)
#         b_train = y_year[:]<=year
#         b_test = y_year[:]>year
#         delete data before year_s
        b_train = (y_year[:]<=year) * (y_year[:]>=year_s)
        b_test = (y_year[:]>year) * (y_year[:]>=year_s)
        x = np.array(self.x)
        y = np.array(self.y_class)
        self.split_xy = (x[b_train,:], y[b_train] , x[b_test,:], y[b_test])
        self.split_xy_count = (x[b_train,:], y_raw[b_train] , x[b_test,:], y_raw[b_test])
    
    def model_build(self, dinp=5):
        x_train, y_train, x_test, y_test = self.split_xy
        model = Sequential()
        layer_0 = Dense(units =self.input_units//dinp, input_dim = self.input_units,
                        kernel_initializer = "random_normal", activation = "relu")
        model.add(layer_0)
        model.add(Dropout(0.35))
        layer_1 = Dense(units =self.input_units//(dinp*5),
                        kernel_initializer = "random_normal", activation = "relu")
        model.add(layer_1)
        model.add(Dropout(0.35))
#         layer_1 = Dense(units =(self.shift1+1),kernel_initializer = "random_normal", activation = "relu")
#         model.add(layer_1)
#         model.add(Dropout(0.25))
        layer_out = Dense(units = self.output_class,kernel_initializer = "random_normal", 
                          activation = "softmax")
        model.add(layer_out)
        #     model.summary()
        model.compile(loss="categorical_crossentropy",optimizer = "adam", metrics = ['accuracy'])
        self.train_model = model

    def model_test(self, epochs=10):
        x_train, y_train, x_test, y_test = self.split_xy
        x_train, x_train_d, y_train, y_train_d = train_test_split(x_train, y_train,  test_size=0.01, shuffle=True)
        train_history = self.train_model.fit(x = x_train, y = np_utils.to_categorical(y_train), 
                                             validation_split = 0.2, epochs =epochs, verbose = 0)
        pre = self.train_model.predict_classes(x_test)
        acc = round(accuracy_score(y_test, pre)*100,2)
        test_result =  (pre , y_test)
        in_acc = round(accuracy_score(self.train_model.predict_classes(x_train), y_train)*100,2)
        train_result = (self.train_model.predict_classes(x_train), y_train)
        return acc,test_result, in_acc,train_result

In [107]:
model_list = glob.glob("./flower_pre_model/04_model/*_15_c*")
m_fn = model_list[0]
# for m_fn in model_list:
flower = m_fn.split("\\")[1].split("_")[0]
station = m_fn.split("\\")[1].split("_")[1]
shift1 =  int(m_fn.split("\\")[1].split("_")[2])
shift2 = 15
y_class_list = [-0.1,0,0.1]
fn_weather = glob.glob("../result/merge_weather/"+ station +"*.csv")[0]
fn_price  = glob.glob("../result/flower_price_byweek/"+ flower + "*.csv")[0]

fp = fp_pre(fn_weather, fn_price)
drop_c = ["year","w_avg", "w_sale", "date", "y_w", "price_diff"]
d_t_dummy = ["market","week"]
fp.all_prep(shift1,shift2,d_t_dummy)
# fp.y_to_class(y_class_list)
# fp.drop_x_data(drop_c)
# model = load_model(m_fn)

# pre  = model.predict_classes(fp.x)
# df = pd.DataFrame()
# df["date"] = fp.d_data
# df["price"]=fp.y_raw_price
# # 先給值再覆蓋
# df["pre_price"]=fp.y_raw_price
# df["pre"]=pre
# df["ans"]=fp.y_class

In [108]:
fp.cy_data.tail(5)

1907    2019-05-27
1908    2019-05-28
1909    2019-05-29
1910    2019-05-30
1911    2019-05-31
Name: date, dtype: object

In [89]:
model_list = glob.glob("./flower_pre_model/04_model/*_15_c*")
# m_fn = model_list[0]
for m_fn in model_list:
    flower = m_fn.split("\\")[1].split("_")[0]
    station = m_fn.split("\\")[1].split("_")[1]
    shift1 =  int(m_fn.split("\\")[1].split("_")[2])
    shift2 = 15
    y_class_list = [-0.1,0,0.1]
    fn_weather = glob.glob("../result/merge_weather/"+ station +"*.csv")[0]
    fn_price  = glob.glob("../result/flower_price_byweek/"+ flower + "*.csv")[0]

    fp = fp_pre(fn_weather, fn_price)
    drop_c = ["year","w_avg", "w_sale", "date", "y_w", "price_diff"]
    d_t_dummy = ["market","week"]
    fp.all_prep(shift1,shift2,d_t_dummy)
    fp.y_to_class(y_class_list)
    fp.drop_x_data(drop_c)
    model = load_model(m_fn)

    pre  = model.predict_classes(fp.x)
    df = pd.DataFrame()
    df["date"] = fp.d_data
    df["price"]=fp.y_raw_price
    # 先給值再覆蓋
    df["pre_price"]=fp.y_raw_price
    df["pre"]=pre
    df["ans"]=fp.y_class
    m_list = ["台中", "台北", "台南", "彰化", "高雄"]
    for m in m_list:
        df[m] = fp.x[m]

    price = np.array(df["price"])
    pre  = np.array(df["pre"])
    pre_price = []

    c = [0.9,0.95,1.05,1.1]
    for i in range(len(price)):
        pre_price.append(price[i]*c[pre[i]])
    df["pre_price"]=pre_price
    df["pre_price"] = df["pre_price"].shift(periods=1)

    arr_m = np.array(df)[:,5:5+5]
    market_list = ["台中","台北","台南","彰化","高雄"]
    m_new = arr_m * market_list
    m_done = []
    for d in m_new:
        m_done.append("".join(d))
    df["market"] = m_done
    df = df.drop(columns=market_list)
    df = df.reset_index(drop=True)

    tags = ["大跌","小跌","小漲","大漲"]
    pre = np.array(df["pre"]).astype(str)
    ans = np.array(df["ans"]).astype(str)
    for i in range(len(tags)):
        pre[pre==str(i)]=tags[i] 
        ans[ans==str(i)]=tags[i]
    df["pre"]=pre
    df["ans"]=ans
    # df.head(20)
    fn_save = "../result/predict_result/" + flower + ".csv"
    df.to_csv(fn_save, encoding="utf-8", index=False)

In [23]:
# model_info = {}
# w_list= glob.glob("../result/merge_weather/*.csv")
# p_list = glob.glob("../result/flower_price_byweek/*")
# select_t = [(1,0),(3,2),(2,1),(2,3),(2,4)]
# st, flower = select_t[3]
# fn_weather = w_list[st]
# fn_price = p_list[flower]
# print(fn_weather, fn_price)
# fp = fp_pre(fn_weather, fn_price)
# drop_c = ["year","w_avg", "w_sale", "date", "y_w", "price_diff"]
# d_t_dummy = ["market","week"]
# # fp.all_prep(12,1,d_t_dummy)
# fp.all_prep(11,1,d_t_dummy)
# fp.y_to_class([-0.1,0,0.1])
# # fp.y_to_class([0])
# fp.drop_x_data(drop_c)
# fp.x.shape,fp.y_yw.shape,fp.y_class.shape
# model = load_model("./flower_pre_model/04_model/OrientalLily_C0K490_11_1_c4.h5")
# # model = load_model("./flower_pre_model/01_model/OrientalLily_C0K490_12_1.h5")
# # x_train, y_train, x_test, y_test = fp.split_xy
# pre  = model.predict_classes(fp.x)
# df = pd.DataFrame()
# df["pre"]=pre
# # df["y_yw"]=fp.y_yw
# df["ans"]=fp.y_class
# df["price"]=fp.y_raw_price
# df["date"] = fp.d_data
# m_list = ["台中", "台北", "台南", "彰化", "高雄"]
# for m in m_list:
#     df[m] = fp.x[m]
# df.head()

# df = pd.DataFrame()
# df["pre"]=pre
# # df["y_yw"]=fp.y_yw
# df["ans"]=fp.y_class
# df["price"]=fp.y_raw_price
# df["date"] = fp.d_data
# m_list = ["台中", "台北", "台南", "彰化", "高雄"]
# for m in m_list:
#     df[m] = fp.x[m]
# # df.to_csv("result_to_plot.csv", index=False)
# # df.to_csv("test0529.csv",index=False)
# price = np.array(df["price"])
# pre  = np.array(df["pre"])
# pre_price = []

# c = [0.9,0.95,1.05,1.1]
# for i in range(len(price)):
#     pre_price.append(price[i]*c[pre[i]])
# df["pre_price"]=pre_price
# df["pre_price"] = df["pre_price"].shift(periods=1)
# # df.head(5)
# arr = np.array(df)
# arr = arr[arr[:,7]==1]
# # arr = arr[-80:]
# df_m_s= pd.DataFrame(arr,columns=df.columns.to_list())
# df_m_s
# # df_new = df_m_s[["price", "pre_price"]]
# # mse = mean_squared_error(df_new["price"], df_new["pre_price"])

# # print(best_mse, best_t)
# # arr = arr[:,0:4]
# # plt.plot(range(len(df_new)),df_new["price"])
# # plt.plot(range(len(df_new)),df_new["pre_price"])
# # df_new.head()

../result/merge_weather\C0K490_古坑.csv ../result/flower_price_byweek\OrientalLily_pbyweek.csv


,pre,ans,price,date,台中,台北,台南,彰化,高雄,pre_price
0,0,0,149.774,2015-07-08,0,0,0,1,0,184.336
1,1,1,135.767,2015-07-15,0,0,0,1,0,134.796
2,2,2,141.801,2015-07-22,0,0,0,1,0,128.978
3,3,3,182.55,2015-07-29,0,0,0,1,0,148.891
4,3,3,223.601,2015-08-05,0,0,0,1,0,200.805
5,2,1,207.095,2015-08-12,0,0,0,1,0,245.961
6,0,0,177.126,2015-08-19,0,0,0,1,0,217.45
7,2,3,195.512,2015-08-26,0,0,0,1,0,159.413
8,0,0,173.732,2015-09-02,0,0,0,1,0,205.287
9,0,1,170.58,2015-09-09,0,0,0,1,0,156.359


In [4]:
# df_m = pd.DataFrame()
# m_list = ["台中", "台北", "台南", "彰化", "高雄"]
# for m in m_list:
#     df[m] = fp.x[m]
# df_m.head()

In [7]:
# df.head(20)
# # m = m_list[0]
# arr = np.array(df)
# # select m 
# arr = arr[arr[:,5]==1]
# arr = arr[:,0:4]
# arr[0:5,:]

In [8]:
# # arr = np.array(df)
# green = arr[arr[:,0]==0]
# red = arr[arr[:,0]==1]
# plt.plot(green[:,3],green[:,2])
# plt.plot(red[:,3],red[:,2])

In [9]:
# model_info = {}
# w_list= glob.glob("../result/merge_weather/*.csv")
# p_list = glob.glob("../result/flower_price_byweek/*")
# select_t = [(1,0),(3,2),(2,1),(2,3),(2,4)]
# st, flower = select_t[3]
# fn_weather = w_list[st]
# fn_price = p_list[flower]
# fp = fp_pre(fn_weather, fn_price)
# model_info["flower"]= fp.flower_name
# model_info["station"]= fp.station_name
# drop_c = ["year","w_avg", "w_sale", "date", "y_w", "price_diff"]
# d_t_dummy = ["market"]
# fp.all_prep(1,1,d_t_dummy)
# fp.dummy

In [10]:
# w_list= glob.glob("../result/merge_weather/*.csv")
# p_list = glob.glob("../result/flower_price_byweek/*")
# # st = 0
# records = pd.DataFrame()
# for st in range(4):
#     fn_weather = w_list[st]
#     st_n  = fn_weather.split("\\")[-1].split("_")[0]
#     fn_price = p_list[0]
#     print(fn_weather, fn_price)
#     fp = fp_pre(fn_weather, fn_price)
#     fp.prep_rdata()
#     records[st_n] = fp.recordz
# records
# records.to_csv("./flower_pre_model/weather_trans.csv", index=False)

In [11]:
# def main_model_run(select,year_split,shift1, shift2, list_split_y):
# #     class_y_diff =[0]
#     model_info = {}
#     w_list= glob.glob("../result/merge_weather/*.csv")
#     p_list = glob.glob("../result/flower_price_byweek/*")
#     select_t = [(1,0),(3,2),(2,1),(2,3),(2,4)]
#     st, flower = select_t[select]
#     fn_weather = w_list[st]
#     fn_price = p_list[flower]
#     fp = fp_pre(fn_weather, fn_price)
#     model_info["flower"]= fp.flower_name
#     model_info["station"]= fp.station_name
#     drop_c = ["year","w_avg", "w_sale", "date", "y_w", "price_diff"]
#     d_t_dummy = ["market", "week"]
#     fp.all_prep(shift1, shift2,d_t_dummy)
# #     fp.y_to_class(list_split_y)
#     fp.y_to_class(list_split_y)
#     fp.drop_x_data(drop_c)
    
#     fp.split_data(year_split)
#     model_info["weather_trans"] = fp.recordz
#     model_info["shift"] = (shift1, shift2)
#     dinp = 5
#     acc = []
#     epo_cf = []
#     acc_max = 0
#     epochs = 3
#     epochs_try =20
#     plot_x = []
#     fp.model_build(dinp)
#     test_acc = []
#     train_acc = []
#     #         print(epochs, end="\t")
#     for i in range(epochs_try):
#         t_acc , test_result, in_acc,train_result= fp.model_test(epochs)
#         acc.append(t_acc)
#         if t_acc>acc_max and in_acc>50:
#             acc_max = t_acc
#             best_test = test_result
#             in_acc_max  = in_acc
#             best_train = train_result
#             best_epochs = epochs*(i+1)
#             best_mode = fp.train_model
#         test_acc.append(t_acc)
#         train_acc.append(in_acc)
#     model_info["acc_info"] = [acc_max, in_acc_max, best_epochs]
# #     model_info.append(best_mode)
#     return model_info, best_mode

In [12]:
# all_model_data =[]
# save_path = "./flower_pre_model/04_model/"
# select = 0
# year_split = 2018
# shift1 = 7
# shift2 = 1

# list_split_y = [-0.1,0,0.1]
# for shift2 in [1,8,15]: # [1,8,15]:
#     for select in range(5): 
#         best_test_acc = 0 
#         for shift1 in range(7,16,2):
#             records , best_model= main_model_run(select,year_split,shift1, shift2, list_split_y)
#             model_fn = save_path + "_".join((records["flower"], records["station"], str(records["shift"][0]),
#                                              str(records["shift"][1]))) + "_c4.h5"
#             model_data  = [records["flower"], records["station"], str(records["shift"][0]),
#                            str(records["shift"][1]), records['acc_info'][0], records['acc_info'][1]]
#             if best_test_acc< records['acc_info'][0]:
#                 best_test_acc = records['acc_info'][0]
#                 best_model_save = best_model
#                 best_model_fn = model_fn
#             all_model_data.append(model_data)
            
#         best_model_save.save(best_model_fn)
# pd.DataFrame(all_model_data, columns=["flower", "station", "shift1", "shift2", "test_acc", "train_acc"])

In [13]:
# # records
# df = pd.DataFrame(all_model_data, columns=["flower", "station", "shift1", "shift2", "test_acc", "train_acc"])
# df.to_csv("./flower_pre_model/04_model/result.csv", index=False, encoding="utf-8")

In [14]:
# df.columns.to_list()

In [15]:
# df = pd.DataFrame(all_model_data, columns=["flower", "station", "shift1", "shift2", "test_acc", "train_acc"])
# g = df.groupby(["flower","shift2"])
# best_data = []
# for flower in ["Anthurium", "Eustoma", "Chrysanthemum","OrientalLily","Rose"]:
#     for sh2 in ["1","8","15"]:
#         df_t = g.get_group((flower,sh2))
#         best_data.append(list(df_t.loc[df_t['test_acc'].idxmax()]))
# df_b = pd.DataFrame(best_data, columns=['flower', 'station', 'shift1', 'shift2', 'test_acc', 'train_acc'])
# df_b

In [16]:
# df_b.to_csv("./flower_pre_model/04_model/result_best.csv", index=False, encoding="utf-8")

In [17]:
# select = 0
# year_split = 2018
# shift1 = 7
# shift2 = 1
# list_split_y = [0]
# model_info = []
# w_list= glob.glob("../result/merge_weather/*.csv")
# p_list = glob.glob("../result/flower_price_byweek/*")
# select_t = [(1,0),(3,2),(2,1),(2,3),(2,4)]
# st, flower = select_t[select]
# fn_weather = w_list[st]
# fn_price = p_list[flower]
# fp = fp_pre(fn_weather, fn_price)
# model_info.append(fp.flower_name)
# drop_c = ["year","w_avg", "w_sale", "date", "y_w", "price_diff"]
# d_t_dummy = ["market", "week"]
# fp.all_prep(shift1, shift2,d_t_dummy)
# fp.y_to_class(list_split_y)
# fp.drop_x_data(drop_c)
# fp.split_data(year_split, year_s)


In [18]:
# main_model_run(select,year_split,shift1, shift2, class_y_diff)


In [19]:
# 花名 測站名稱 shift (參數,預測週(1,8,15), 準確率範圍)
# Anthurium 467480     shift:(14,1,55~63%), (27,8,46~65%) (19,15,55~72%)
# Chrysanthemum C0F9L0 shift:(19,1,51~58%), (16,8,40~59%) (16,15,33~50%)
# Eustoma C0F9L0       shift:(3,1,67~73%) (16,8,49~57%) (12,15,51~63%)  epochs [10,]
# OrientalLily C0X060  shift:(17,1,77~79%) (22,8,66~75%) (14,15,43~64%) epochs [ , 12, ]
# Rose C0X060          shift:(7,1,47~64%) (18,8,42~64%) (18,15,55~69%)